# CS614 Assignment 1 - LLM Training Code

In [30]:
!pip install transformers evaluate datasets
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00


In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import evaluate, torch
from datasets import load_dataset, get_dataset_split_names

## **Dataset**:
The dataset is obtained from https://huggingface.co/datasets/abisee/cnn_dailymail. It contains more than three thousand news articles from CNN and Daily Mail, and used for extractive and abstractive summarisation.

## **Task:**
Summarise news articles using the selected LLM.

In [11]:
#Load dataset
ds = load_dataset("abisee/cnn_dailymail", "1.0.0") #config 1
#Get split names
get_dataset_split_names("abisee/cnn_dailymail", "1.0.0")

1.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

1.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

1.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

1.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

1.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

['train', 'validation', 'test']

In [18]:
#Check if dataset extracted successfully
ds["train"][15]

{'article': 'WASHINGTON (CNN) -- There is "no remaining hope" of finding six men trapped for almost a month in a Utah coal mine alive, a federal official said Saturday. Isaac Arellano holds a candle and sings during a fundraiser for miners Tuesday in Price, Utah. "Over the past 25 days, the Mine Safety and Health Administration has exhausted all known options in our attempt to reach the six miners," Richard Stickler, head of the agency, said in a statement. "The thoughts and prayers of the dedicated professionals at MSHA are with the families." Sympathy for the failed efforts also came Saturday from the White House. "Last night, a difficult decision was made to end the search," President Bush said in a statement. "Laura and I are deeply saddened by this tragedy and continue to pray for the families of these men." Labor Secretary Elaine Chao called the ordeal "heartbreaking." "The grueling around-the-clock rescue operation that claimed three lives and injured six others has also taken a

In [16]:
#check the attributes (features) of dataset
ds["train"].features

{'article': Value('string'),
 'highlights': Value('string'),
 'id': Value('string')}

`article`: Original article
<br>`highlights`: Summary reference

## **Model:**


In [38]:
model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda() #set to GPU
bert_score = evaluate.load("bertscore")

In [39]:
idx = 157
sample = ds["train"][idx]["article"]
sample_result = ds["train"][idx]["highlights"]

In [40]:
print(f"Original text: {sample}\nSummary: {sample_result}")

Original text: ROME, Italy (CNN) -- The garbage crisis in Naples encompasses the worst Italian clichés, and in particular those of the southern part of this lovely peninsula: mismanagement, political interference, mafia profiteering and the ability of those responsible to deflect the attention and the blame elsewhere. Naples has had problems in finding sites for municipal dumps -- now workers have stopped collecting trash. There is a popular saying here that roughly goes like this: everybody is competent enough (to find a solution) but nobody is responsible (for actually carrying it out). In many parts of the world waste disposal is a business -- and usually it is a good business. Garbage can be transformed into various sources of energy and then sold for a profit. In Naples, garbage is also good business, but in the sense that millions, if not billions, of euros have been wasted -- and nobody really knows how. The problem is as old and ugly as rotten trash. The region's dumps reached 

In [41]:
with torch.no_grad():
  message = f"Please summarise this into one paragraph: {sample}\n"
  tokens = tokenizer.encode(message, return_tensors="pt").to("cuda")
  response = model.generate(tokens, max_new_tokens=1024, use_cache=True)

print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Please summarise this into one paragraph: ROME, Italy (CNN) -- The garbage crisis in Naples encompasses the worst Italian clichés, and in particular those of the southern part of this lovely peninsula: mismanagement, political interference, mafia profiteering and the ability of those responsible to deflect the attention and the blame elsewhere. Naples has had problems in finding sites for municipal dumps -- now workers have stopped collecting trash. There is a popular saying here that roughly goes like this: everybody is competent enough (to find a solution) but nobody is responsible (for actually carrying it out). In many parts of the world waste disposal is a business -- and usually it is a good business. Garbage can be transformed into various sources of energy and then sold for a profit. In Naples, garbage is also good business, but in the sense that millions, if not billions, of euros have been wasted -- and nobody really knows how. The problem is as old and ugly as rotten trash. 

## Evaluation: BERTScore
Even though there is a reference text that can be used for ROUGE Evaluation, I decided to use BERTScore as it can detect paraphrasing and semantic similarity of the reference text and their corresponding summary, which will be a more accurate representation of the model's performance.